## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
del city_data_df['Unnamed: 0']
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Sydney Mines,CA,46.24,-60.22,70.25,87,75,12.66,broken clouds
1,Kodiak,US,57.79,-152.41,49.51,82,7,3.33,clear sky
2,Puerto Ayora,EC,-0.74,-90.35,67.55,100,100,7.00,overcast clouds
3,Bucak,TR,37.46,30.59,88.32,36,62,6.87,broken clouds
4,Haines Junction,CA,60.75,-137.51,37.35,87,92,3.44,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your trip? "))
max_temp=float(input("What is the maximum temerpature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temerpature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & (city_data_df["Max Temp"]>= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Sydney Mines,CA,46.24,-60.22,70.25,87,75,12.66,broken clouds
8,Banjar,ID,-8.19,114.97,78.44,87,100,2.57,overcast clouds
9,Grand Gaube,MU,-20.01,57.66,73.80,71,3,8.99,clear sky
12,Walvis Bay,NaN,-22.96,14.51,76.15,43,0,18.41,clear sky
19,Nhulunbuy,AU,-12.23,136.77,70.41,88,19,6.91,few clouds
21,Hilo,US,19.73,-155.09,77.29,59,2,4.61,clear sky
24,Mogadishu,SO,2.04,45.34,80.47,74,75,18.41,broken clouds
26,Harper,LR,4.38,-7.72,78.98,79,71,9.15,light rain
27,Souillac,MU,-20.52,57.52,73.78,73,40,12.66,scattered clouds
28,Varna,BG,43.22,27.92,71.74,71,40,11.50,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   273
Country                271
Lat                    273
Lng                    273
Max Temp               273
Humidity               273
Cloudiness             273
Wind Speed             273
Current Description    273
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_preferred_cities_df = preferred_cities_df.dropna()
new_preferred_cities_df.count()


City                   271
Country                271
Lat                    271
Lng                    271
Max Temp               271
Humidity               271
Cloudiness             271
Wind Speed             271
Current Description    271
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sydney Mines,CA,70.25,broken clouds,46.24,-60.22,
8,Banjar,ID,78.44,overcast clouds,-8.19,114.97,
9,Grand Gaube,MU,73.80,clear sky,-20.01,57.66,
19,Nhulunbuy,AU,70.41,few clouds,-12.23,136.77,
21,Hilo,US,77.29,clear sky,19.73,-155.09,
24,Mogadishu,SO,80.47,broken clouds,2.04,45.34,
26,Harper,LR,78.98,light rain,4.38,-7.72,
27,Souillac,MU,73.78,scattered clouds,-20.52,57.52,
28,Varna,BG,71.74,scattered clouds,43.22,27.92,
30,Rikitea,PF,71.42,overcast clouds,-23.12,-134.97,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found....skipping")
        

Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   271
Country                271
Max Temp               271
Current Description    271
Lat                    271
Lng                    271
Hotel Name             271
dtype: int64

In [15]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sydney Mines,CA,70.25,broken clouds,46.24,-60.22,At The Harbourfront B&B Inc
8,Banjar,ID,78.44,overcast clouds,-8.19,114.97,The Hamsa Resort
9,Grand Gaube,MU,73.80,clear sky,-20.01,57.66,Veranda Paul et Virginie Hotel & Spa
19,Nhulunbuy,AU,70.41,few clouds,-12.23,136.77,East Arnhem Real Estate
21,Hilo,US,77.29,clear sky,19.73,-155.09,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
667,Danville,US,74.77,overcast clouds,36.59,-79.39,"Holiday Inn Express Danville, an IHG Hotel"
670,Jiuquan,CN,70.84,clear sky,39.74,98.52,Jiuquan Hotel
671,Kilindoni,TZ,76.64,few clouds,-7.91,39.67,Bustani B&B
678,Glendive,US,74.17,clear sky,47.11,-104.71,Super 8 by Wyndham Glendive


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv" 
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template= """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weahter</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
current_weather = hotel_df["Current Description"]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(40.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
plt.savefig("WeatherPy_vacation.map.png")

NameError: name 'plt' is not defined